<a href="https://colab.research.google.com/github/pedroconcejero/deep_learning_CNN/blob/main/Paquita_Salas_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importación de Librerias**

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Descarga y preprocesado de datos**

In [5]:
#
text = open("/content/gdrive/MyDrive/Colab Notebooks/Paquita_Salas.txt", 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:        {} carácteres'.format(len(text)))

vocab = sorted(set(text))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

Longitud del texto:        84996 carácteres
El texto está compuesto de estos 100 carácteres:
['\n', '\r', ' ', '!', '#', '%', '&', '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', ':', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '´', '¿', 'Á', 'É', 'Í', 'Ñ', 'Ó', 'Ú', 'Ü', 'á', 'ç', 'é', 'í', 'ñ', 'ó', 'ú', 'ü', '’', '“', '”', '…', '\ufeff']


In [6]:
#Como las redes neuronales solo procesan valores numéricos, no letras. Traduciremos los caracteres a representación numérica.
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '#' :   4,
  '%' :   5,
  '&' :   6,
  '(' :   7,
  ')' :   8,
  ',' :   9,
  '-' :  10,
  '.' :  11,
  '/' :  12,
  '0' :  13,
  '1' :  14,
  '2' :  15,
  '3' :  16,
  '4' :  17,
  '5' :  18,
  '6' :  19,
  '7' :  20,
  ':' :  21,
  '?' :  22,
  '@' :  23,
  'A' :  24,
  'B' :  25,
  'C' :  26,
  'D' :  27,
  'E' :  28,
  'F' :  29,
  'G' :  30,
  'H' :  31,
  'I' :  32,
  'J' :  33,
  'K' :  34,
  'L' :  35,
  'M' :  36,
  'N' :  37,
  'O' :  38,
  'P' :  39,
  'Q' :  40,
  'R' :  41,
  'S' :  42,
  'T' :  43,
  'U' :  44,
  'V' :  45,
  'W' :  46,
  'X' :  47,
  'Y' :  48,
  'Z' :  49,
  '_' :  50,
  'a' :  51,
  'b' :  52,
  'c' :  53,
  'd' :  54,
  'e' :  55,
  'f' :  56,
  'g' :  57,
  'h' :  58,
  'i' :  59,
  'j' :  60,
  'k' :  61,
  'l' :  62,
  'm' :  63,
  'n' :  64,
  'o' :  65,
  'p' :  66,
  'q' :  67,
  'r' :  68,
  's' :  69,
  't' :  70,
  'u' :  71,
  'v' :  72,
  'w' :  73,
  'x' :  74,
  'y' :  75,
  'z' :  76,

In [7]:
#Ahora pasaremos el texto a un array de enteros
text_as_int = np.array([char2idx[c] for c in text])

In [8]:
#Mostramos los 50 primeros caracteres del texto text_as_init
print ('texto : {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto : '\ufeffPAQUITA:\r\nHola, soy Paquita Salas y soy represent'
array([99, 39, 24, 40, 44, 32, 43, 24, 21,  1,  0, 31, 65, 62, 51,  9,  2,
       69, 65, 75,  2, 39, 51, 67, 71, 59, 70, 51,  2, 42, 51, 62, 51, 69,
        2, 75,  2, 69, 65, 75,  2, 68, 55, 66, 68, 55, 69, 55, 64, 70])


**Preparación de los datos para entrenar la RNN**

In [9]:
#Para entrenar el modelo creamos un conjunto de datos con el contenido de text_as_init. Para ello utilizamos la función tf.data.Dataset.from_tensor_slices.
#A este conjunto de datos lo dividiremos en secuencias de seq_length+1 al aplicar el método batch()

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [10]:
#Comprobamos que las sequences son el texto dividido en 101 caracteres (mostramos 10 secuencias)
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'\ufeffPAQUITA:\r\nHola, soy Paquita Salas y soy representante.\r\n\r\n\r\nPAQUITA SALAS habla acaloradamente por t'
'eléfono. \r\n\r\n\r\nUn cuadro gigante de MACARENA GARCÍA, su actriz más importante, corona la sala. Al lad'
'o del cuadro, una estantería gigante llena de videobooks de actores. \r\n\r\n\r\nPAQUITA: \r\n¡¡Pero el acent'
'o colombiano, ella se trabaja el acento colombiano!! \r\n\r\n\r\nMAGÜI, una mujer de 30 años, andaluza, alt'
'a y con el pelo corto, se acerca a ella temerosa. \r\n\r\n\r\nMAGÜI: \r\nPaquita… \r\n\r\n\r\nPAQUITA:\r\n¿Qué? \r\n\r\n\r'
'\nMAGÜI: \r\nEstá aquí Lidia. \r\n\r\n\r\nLIDIA SAN JOSÉ, una actriz representada por PAQUITA, está en la puer'
'ta de la oficina. Saluda, tímida. PAQUITA, al teléfono, le lanza un beso con la mano. \r\n\r\n\r\nPAQUITA ('
'A MAGÜI):\r\nLo de Oysho está ahí. (Al teléfono) ¡¡Es actriz, ella se ha hecho cursos!! \r\n\r\n\r\nMAGÜI cog'
'e una bolsa y se la da a LIDIA, que le da las gracias. LIDIA saca de la bolsa un camisón. 

In [11]:
#Creamos una función que devolverá el conjunto de datos de entrenamiento (los datos de entrada como los datos de salida)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

#Aplicamos la función a todas las secuencias utilizando el método map()
dataset = sequences.map(split_input_target)

In [12]:
#Los dataset contienen un conjunto de parejas (100 caracteres del texto original, la correspondiente salida ). Vamos a mostrar la primera pareja.
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

  print(dataset)

Input data:  '\ufeffPAQUITA:\r\nHola, soy Paquita Salas y soy representante.\r\n\r\n\r\nPAQUITA SALAS habla acaloradamente por '
Target data: 'PAQUITA:\r\nHola, soy Paquita Salas y soy representante.\r\n\r\n\r\nPAQUITA SALAS habla acaloradamente por t'
<_MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>


In [13]:
#Agrupamos los dataset en batches de 64 . Así tendriamos los datos de entrenamiento con batches compuestos de 64 parejas de secuencias de 100 integers de 64 bits
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


**Construcción del modelo RNN**

In [24]:
#Crearemos una función que cree un modelo RNN con tres capas
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  #Añadimos la capa de tipo word embedding
  model.add(Embedding(input_dim=vocab_size,
                      output_dim=embedding_dim,
                      #batch_input_shape=[batch_size, None] Deprecated
                      ))
  #Añadimos la capa de tipo LSTM
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(512, activation="relu"))
  model.add(Dropout(0.5))


  #Añadimos la capa de tipo Dense
  model.add(Dense(vocab_size))
  return model

In [25]:
#Creamos el modelo
embedding_dim = 256
rnn_units = 1024

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [26]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [27]:
#A continuación inspecionamos las dimenciones de los tensores
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")

Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print("Prediction : ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction :  (64, 100, 100) # (batch_size, sequence_length, vocab_size)


In [29]:
#Obtenemos una muestra de la distribución de salida
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [30]:
print(sampled_indices_characters)

[82 63 38 69  6 12 26 78 13 33 99 90 84 77 51 88 31 76 84 34 46 72  2 91
 87 62 71  2 28 38 57 66 73 44 86  6 57 55 42  4 72 91 30 19 10  3 35 71
 18 57 93 67 11 47 30 21 16 29 69 14 89 82 82 21 56 71 34  9 31 30 49 56
 19 33 37 63  5 14 23 18 71 58 55 96 87 59 89 21 29 72 22 41 21 73 42 22
  5 46 65 34]


**Entrenamiento del modelo RNN**

In [31]:
#Creamos la función de perdida, usaremos el categorical pues estamos considerando datos categóricos
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [32]:
#Compilamos el modelo
model.compile(optimizer='adam', loss=loss)

In [33]:
#configuramos los checkpoints

checkpoint_dir = './training_checkpoints_Paquita'

# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [34]:
#Entrenamos el modelo
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 94ms/step - loss: 4.1522
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - loss: 3.3155
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - loss: 2.8902
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - loss: 2.5460
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - loss: 2.3300
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - loss: 2.2127
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - loss: 2.1022
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - loss: 2.0553
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - loss: 1.9936
Epoch 10/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 1.9587
Epoch 11/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - loss: 1.9030
Epoch 12/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - loss: 1.8509
Epoch 13/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - loss: 1.8205
Epoch 14/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 1.7623
Epoch 15/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - loss: 1.7296
Epo

In [35]:
model.save("model_paquita_100_2024.keras")

In [37]:
from keras.models import load_model
from keras import losses # Import the losses module

# Assuming your original loss function was, for example, binary_crossentropy
loaded_model = load_model("model_paquita_100_2024.keras",
                          custom_objects={'loss': losses.sparse_categorical_crossentropy})
# or if it was a custom loss function
# loaded_model = load_model("model_paquita_100_2024.keras", custom_objects={'loss': my_custom_loss_function})

**Generación de texto del modelo RNN**

In [43]:
#Ya que tenemos entrenado los modelos vamos a usarlos para generar texto
#Vamos a reconstruir manualmente los modelos para modifical el batch y su peso para poner el último
model = build_model(len(vocab), embedding_dim, rnn_units, batch_size=1)
# Build the model before loading weights
# This creates the necessary layer variables to which weights can be assigned
# Provide an input shape that matches the expected input of your model
# for example:
# input_shape = (1, sequence_length) where sequence_length is the length of your input sequences
input_shape = (1, 100)  # Replace 100 with your actual sequence length
model.build(input_shape=input_shape) # Or model.build(tf.TensorShape([1, None]))

model.load_weights("model_paquita_100_2024.keras")

In [44]:
#Creamos una función generar_texto que generará texto a partir de una palabra de partida
def generate_text(model, start_string):

  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  temperature = 0.7

#  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)

      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [45]:
print(generate_text(model, start_string=u"Paquita espera en el coche"))

Paquita espera en el coche. Está pensativa. Llega MAGÜI con la mano y MAGÜI sonríe. PAQUITA y MAGÜI se abrazan en la puerta de un cine. Vemos que le he mirado en descocharlos. MARIONA se gonta en el coche y se aleja de ver, por un dos mánda, Puedo ser tu esclava, Puedo ser tu diva, Puedo ser tu esclava, Puedo ser tu diva, Puedo ser tu secretaria pero que no sabe ni que yo estoy allí. Detrás de todo. 


PAQUITA se pone el abrigo. 


MAGÜI le traba a la cama. 


PAQUITA: 
Ya te lo he dicho. Porque a ver si puedes arrasar al entrar se vente en cositente. Al hara del baz, se lova la mano y un cambio de la calera, que estaba a puerta de que llamas perplea de casa. PAQUITA se despierta, se levanta y quita, encanta. 








































































La puerta de PS MANAGEMENT se abre. Entra PAQUITA con gafas de sol, se encuentra de frente con MAGÜI intentando descolgar el cartel gigante de MACARENA


In [46]:
print(generate_text(model, start_string=u"qué es spam"))

qué es spamena con micasona. 


MAGÜI coge su abrigo y entra detrás de la barra. 


PAQUITA (Al teléfono): 
¿Huanfran? Nene, que soy yo. Oye, que ya nos vamos a la boda. 


MAGÜI: 
Es que estos videobooks de parada. 


PAQUITA: 
¡Ay! Hija, ella está perdigo. 


MAGÜI: 
¿Qué tal la reconica y MAGÜI le da la récina. PAQUITA se despenta, entra de es que no me lo cuenta nada y que no esa haciendo La nieta de Mariona Felga. 


PAQUITA: 
¡Hija, estamos en comecirado estos videobooks de actores y actrices se queda sola. 


PAQUITA: 
Sí. Te dejo, expañano que te sale aquí la Estrella. A ver si está fuera la escena y BELEX la está antiendo en comecia a tu casa. ¡Hoy nos cierras la ofcina! Desflan de un lado a otro por turnos. MAGÜI se le da la rapais. MAGÜI le del puedio, andiestradera que tergan una copienta de ligar en colos. 


MAGÜI se queda paralizada. PAQUITA ostá fomo lo tara a MAGÜI con la mano y MAGÜI sonríe. PAQUITA y MAGÜI se abrazan en la puerta de una caleta de M
